In [6]:
import pandas as pd
from langchain import PromptTemplate
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI
import warnings
from langchain.chains import LLMChain
warnings.filterwarnings("ignore")

In [21]:
data = pd.read_csv("../Resources/Raw/air_pollution.csv")
print("Cantidad de registros:",str(data.shape[0]))
data.head(5)


Cantidad de registros: 6985


,city,country,2017,2018,2019,2020,2021,2022,2023
0,Kabul,Afghanistan,0,61.8,58.8,46.5,37.5,17.1,18.1
1,Tirana,Albania,0,0,0,16.0,12.5,14.5,14.4
2,Algiers,Algeria,0,0,21.2,20.2,20.0,17.8,17.4
3,Ordino,Andorra,0,0,0,7.4,7.3,5.4,5.3
4,Luanda,Angola,0,0,15.9,13.0,11.0,8.8,8.7


In [7]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model = AzureChatOpenAI(temperature=0, deployment_name="chat")

In [44]:
template = """
You have the following information {context} which contains information about the population in millions of each country and its corresponding city
and you must answer the following question {question}
Limit yourself to delivering the answer.
Response in spanish
"""

prompt = PromptTemplate(
    template = template,
    input_variables = ["context","question"]
)

llm = LLMChain(llm=model, prompt=prompt)

llm.run(context = data,
         question = "Que año se tuvo mayor cantidad de población? y con cuanto fue?")




'El año con la mayor cantidad de población fue el año 2017, con una población de 61.8 millones.'